In [ ]:
import pickle
import numpy as np
import holoviews as hv
from holoviews import opts
hv.extension('bokeh', 'matplotlib')

down_only_segmented = pickle.load(open('./data/down_only_segmented.pickle','rb'))
pushed_down_only_segmented = pickle.load(open('./data/pushed_down_only_segmented.pickle','rb'))
down_up_segmented = pickle.load(open('./data/down_up_segmented.pickle','rb'))
pushed_down_up_segmented = pickle.load(open('./data/pushed_down_up_segmented.pickle','rb'))
robot_hand_segmented = pickle.load(open('./data/robot_hand_segmented.pickle','rb'))
down_up_lt = pickle.load(open('./data/down_up_lt.pickle','rb'))
down_only_lt = pickle.load(open('./data/down_only_lt.pickle','rb'))
pushed_down_up_lt = pickle.load(open('./data/pushed_down_up_lt.pickle','rb'))
pushed_down_only_lt = pickle.load(open('./data/pushed_down_only_lt.pickle','rb'))


def get_vlines(cut_point):
    try:
        vlines = hv.VLine(cut_point[0])
        for cp in cut_point[1:]: vlines *= hv.VLine(cp)
    except: pass
    try: return(vlines)
    except: return(hv.VLine(-1.0))
def plot_data(d, cut_point):
    return((get_vlines(cut_point)
            *hv.Curve(d[0],'step','action',label='x_acc')*hv.Curve(d[1],'step','action',label='y_acc')
            *hv.Curve(d[2],'step','action',label='action_on')*hv.Curve(d[3],'step','action',label='action_off')))
def plot_hand(d, cut_point):
    return((get_vlines(cut_point)
            *hv.Curve(d[0],'step','action',label='x_vel')*hv.Curve(d[1],'step','action',label='y_vel')
            *hv.Curve(d[2],'step','action',label='rotation')*hv.Curve(d[3],'step','action',label='grasp')
            *hv.Curve(d[4],'step','action',label='action_on')*hv.Curve(d[5],'step','action',label='action_off')))
def plot_segment(dist, cut_point):
    point = [(cp, dist[cp]) for cp in cut_point]
    return((hv.Curve(dist,'step','distance',label='distance'))
           *hv.Scatter(point,'step','distance',label='segmentation point')
           *get_vlines(cut_point))

def plot(data, dtype):
    if dtype == 'ContWorld':plot_func = plot_data
    else: plot_func = plot_hand
    xs = data['xs']
    dist = data['dists']
    cut_point = data['cut_points']
    return(hv.HoloMap({i: hv.Layout(plot_func(xs[i], cut_point[i]) + plot_segment(dist[i], cut_point[i])).cols(1) for i in range(len(xs))}, 'Index'))

## 1 Expert actions
This section exhibits all expert actions. You can check full dataset by sliding index.

### 1.1 ContinuousWorld
#### 1.1.1 Down&Up

In [ ]:
%opts Curve [width=600 height=250] Overlay [legend_position='right']
%opts Curve (alpha=0.5)
%opts Scatter (color='purple', size=10, alpha=0.5)
%opts VLine (color='purple', line_alpha=0.5)
plot(down_up_segmented, 'ContWorld')

#### 1.1.2 DownOnly

In [ ]:
%opts Curve [width=600 height=250] Overlay [legend_position='right']
%opts Curve (alpha=0.5)
%opts Scatter (color='purple', size=10, alpha=0.5)
%opts VLine (color='purple', line_alpha=0.5)
plot(down_only_segmented, 'ContWorld')

#### 1.1.3 PushedDown&Up

In [ ]:
%opts Curve [width=600 height=250] Overlay [legend_position='right']
%opts Curve (alpha=0.5)
%opts Scatter (color='purple', size=10, alpha=0.5)
%opts VLine (color='purple', line_alpha=0.5)
plot(pushed_down_up_segmented, 'ContWorld')

#### 1.1.4 PushedDownOnly

In [ ]:
%opts Curve [width=600 height=250] Overlay [legend_position='right']
%opts Curve (alpha=0.5)
%opts Scatter (color='purple', size=10, alpha=0.5)
%opts VLine (color='purple', line_alpha=0.5)
plot(pushed_down_only_segmented, 'ContWorld')

### 1.2 RobotHand

In [ ]:
%opts Curve [width=600 height=250] Overlay [legend_position='right']
%opts Curve (alpha=0.5)
%opts Scatter (color='purple', size=10, alpha=0.5)
plot(robot_hand_segmented, 'RobotHand')

## 2 Disentangled representation learning
This section exhibits interactive visualization of latent traversal. You can slide parameters to change value of only one latent variable, which changes the reconsturcted output. First plot exhibits original action, second one exhibits reconstruct of the original action by FAVAE, and last one exhibits latent traversal.

You can change three parameters: *Ladder Index*, *Z Index*, and *Z Value*. *Ladder Index* and *Z Index* choose which latent variable to change and *Z Value* changes value of the chosen latent variable.

\* Any *Z Index* that exceeds maximum number of Z in each ladder is invalid; i.e, *Z Index*=1 in *Ladder Index*=2 is invalid and just follows plots of previous *Z Index*, because that *Z Index* does not exist.

\* Maximum number of *Z Index* of each ladder is: 4 for ladder 0, 2 for ladder 1, and 1 for ladder 2.

In [ ]:
data_idx_range = [0]
#ladder_idx_range = [0,1,2]
#z_idx_range_cont = [[0,1,2,3],[0,1],[0]]
#z_idx_range_robot = [[0,1,2,3,4,5,6,7,8,9,10,11],[0,1,2,3,4,5,6,7],[0,1,2,3]]
ladder_idx_range = [1,2,3]
z_idx_range_cont = [[1,2,3,4],[1,2],[1]]
z_idx_range_robot = [[1,2,3,4,5,6,7,8,9,10,11,12],[1,2,3,4,5,6,7,8],[1,2,3,4]]
z_idx_max_cont = 4
z_idx_max_robot = 12
Z_MIN = -3.0
Z_MAX = 3.0
PLOT_Z_NUM = 13 
z_vals = [Z_MIN+(Z_MAX-Z_MIN)/(PLOT_Z_NUM-1)*i for i in range(PLOT_Z_NUM)]
data_idx_carrier = np.asarray(data_idx_range)
ladder_idx_carrier = np.asarray(ladder_idx_range)
z_idx_carrier = np.asarray(list(range(1,5)))
z_val_carrier = z_vals

def reconstruct(datazip):
    shifted_data = [[[[] for k in z_idx_range_cont[j-1]] for j in ladder_idx_range] for i in data_idx_range]
    for d in datazip:
        data_idx=d['data_idx']
        if data_idx in data_idx_range:
            z_vals=d['z_vals']
            ladder_idx=d['ladder_idx']
            z_idx=d['z_idx']
            data=d['data']
            recon=d['recon']
            shifted_recons=d['shifted_recons']
            shifted_data[data_idx][ladder_idx][z_idx] = shifted_recons
    return(data, recon, shifted_data)


%output max_frames=2000
def plot_(sr,title,null=False):
    if not null:
        lt = hv.Curve(sr[0],'step',title,label='x_acc')*hv.Curve(sr[1],'step',title,label='y_acc')*hv.Curve(sr[2],'step',title,label='action_on')*hv.Curve(sr[3],'step',title,label='action_off')
    else:
        null_sr = np.zeros_like(sr)
        lt = hv.Curve(null_sr[0],'step',title,label='x_acc')*hv.Curve(null_sr[1],'step',title,label='y_acc')*hv.Curve(null_sr[2],'step',title,label='action_on')*hv.Curve(null_sr[3],'step',title,label='action_off')
    return(lt)

def plot_lt(data, recon, shifted_data):
    curve_dict = {(
                   ladder_idx, 
                   z_idx, 
                   z_val
                   ): plot_(shifted_data\
                                 [0]\
                                 [ladder_idx-1]\
                                 [min(z_idx-1,len(z_idx_range_cont[ladder_idx-1])-1)]\
                                 [z_val_carrier.index(z_val)-1][0],
                            title='latent traversal',
                            null=True if z_idx-1 > len(z_idx_range_cont[ladder_idx-1])-1 else False
                            ) 
                for ladder_idx in ladder_idx_carrier 
                for z_idx in z_idx_carrier
                for z_val in z_val_carrier}
    kdims = [
            hv.Dimension(('ladder_idx_carrier', 'Ladder Index'), default=1),
            hv.Dimension(('z_idx_carrier', 'Z Index'), default=1),
            hv.Dimension(('z_val_carrier', 'Z Value'), default=0)]
    org = plot_(data[0],'original action')
    rec = plot_(recon[0],'reconstruction')
    hm = org + rec + hv.HoloMap(curve_dict, kdims=kdims)
    return(hm.cols(1))

### 2.1 Down&Up

In [ ]:
%opts Curve [width=600 height=150] Overlay [legend_position='right']
%opts Curve (alpha=0.5)
data, recon, shifted_data = reconstruct(down_up_lt)
plot_lt(data, recon, shifted_data)

### 2.2 DownOnly

In [ ]:
%opts Curve [width=600 height=150] Overlay [legend_position='right']
%opts Curve (alpha=0.5)
data, recon, shifted_data = reconstruct(down_only_lt)
plot_lt(data, recon, shifted_data)

### 2.3 PushedDown&Up

In [ ]:
%opts Curve [width=600 height=150] Overlay [legend_position='right']
%opts Curve (alpha=0.5)
data, recon, shifted_data = reconstruct(pushed_down_up_lt)
plot_lt(data, recon, shifted_data)

### 2.4 PushedDownOnly

In [ ]:
%opts Curve [width=600 height=150] Overlay [legend_position='right']
%opts Curve (alpha=0.5)
data, recon, shifted_data = reconstruct(pushed_down_only_lt)
plot_lt(data, recon, shifted_data)

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show_err=true; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')